In [29]:
from Backtester.database.candles import CandlesDb
import Backtester.time.time_utility as timly
from peewee import fn
cdb = CandlesDb()

In [30]:
db = cdb.db

In [7]:
bet = (14071.0, 18831.0)
start = timly.to_tms("2022-10-20")
end = timly.to_tms("2022-12-09")
dbAlias = db.alias()
sub_query = dbAlias.select(fn.MIN(dbAlias.timestamp)).where(dbAlias.exchange == "bybit", dbAlias.symbol == "BTC/USDT:USDT", dbAlias.timeframe == "1m", dbAlias.timestamp.between(start, end), dbAlias.low < bet[0])
query = (db.select(fn.MIN(db.timestamp)).where(db.high >= bet[1], db.timestamp.between(sub_query, end)))


In [ ]:
def query_order_pair(db, bet: tuple, start, end):
    dbAlias = db.alias()
    sub_query = dbAlias.select(fn.MIN(dbAlias.timestamp)).where(dbAlias.exchange == "bybit", dbAlias.symbol == "BTC/USDT:USDT", dbAlias.timeframe == "1m", dbAlias.timestamp.between(start, end), dbAlias.low < bet[0])
    query = (db.select(fn.MIN(db.timestamp)).where(db.high >= bet[1], db.timestamp.between(sub_query, end)))
    return sub_query.scalar(), query.scalar()


In [8]:
print(sub_query.scalar(), query.scalar())

None None


# Query Gaps in Database

1. create a Gap

In [2]:
# Creating a Gap
db = cdb.db
dbAlias = db.alias()
query = dbAlias.select(fn.MIN(dbAlias.timestamp)).where(dbAlias.exchange == "bybit", dbAlias.symbol == "BTC/USDT:USDT", dbAlias.timeframe == "1m")

In [7]:
import pandas as pd
str(pd.to_datetime(query.scalar(), unit="ms"))

'2022-09-30 15:41:00'

In [2]:
from Backtester.streamer.candles import streamer

res = streamer.get("bybit", "BTC/USDT:USDT", "4h", start="2022-05-20", end="2022-12-12")

{'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '4h', 'start': 1653004800000, 'end': 1670803200000, 'limit': 1236}


In [20]:
res

({'exchange': 'bybit',
  'market': 'BTC/USDT:USDT',
  'timeframe': '4h',
  'start': 1653004800000,
  'end': 1670803200000,
  'limit': 1236},
 [])

In [9]:
import pandas as pd
st = (1664582400000,1665774900000)

In [16]:
from Backtester.database.helper import _get_bounds_sorted

_get_bounds_sorted(streamer.db.db, {'exchange': 'bybit', 'market': 'BTC/USDT:USDT', 'timeframe': '4h', 'start': 1664582100000, 'end': 1665775000000, 'limit': 1325})

{('s', 'e'): (1664582100000, 1665775000000)}

In [49]:
start, end = timly.safe_tms("2022-09-22"), timly.safe_tms("2022-10-01")
mins = db.select(fn.MIN(db.timestamp)).where(db.exchange == "bybit", db.symbol == "BTC/USDT:USDT", db.timeframe == "1m").scalar()
mins

1663804860000

In [50]:
pd.to_datetime(mins, unit="ms")

Timestamp('2022-09-22 00:01:00')

In [28]:
def find_gaps(db, start, end, di = 60000):
    _db = db.alias()
    ntms = _db.timestamp + di
    ptms = _db.timestamp - di
    subq = (_db.select(_db.timestamp.alias("tms"), ntms.alias("ntms"), ptms.alias("ptms")).where(_db.timestamp.between(start+di, end-di)))

    topq = db.select(db.timestamp)
    query = db.select(subq.c.tms).from_(subq).where((subq.c.ntms.not_in(topq)) | (subq.c.ptms.not_in(topq)))
    return list(query.tuples())


In [51]:
def _find_gaps(db, start, end, di = 60000):
    _db = db.alias()
    ntms = _db.timestamp + di
    ptms = _db.timestamp - di
    subq = (_db.select(_db.timestamp.alias("tms"), ntms.alias("ntms"), ptms.alias("ptms")).where(_db.exchange == "bybit", _db.symbol == "BTC/USDT:USDT", _db.timeframe == "1m", _db.timestamp.between(start+di, end-di)).order_by(_db.timestamp.asc()))

    topq = db.select(db.timestamp).where(db.exchange == "bybit", db.symbol == "BTC/USDT:USDT", db.timeframe == "1m")
    query = db.select(subq.c.tms).from_(subq).where((subq.c.ptms.not_in(topq)) | (subq.c.ntms.not_in(topq)))
    return list(query.tuples())

In [98]:
def find_gaps_sorted(db, start, end, di = 60000):
    _db = db.alias()
    ntms = _db.timestamp + di
    ptms = _db.timestamp - di
    subq = (_db.select(_db.timestamp.alias("tms"), ntms.alias("ntms"), ptms.alias("ptms")).where(_db.exchange == "bybit", _db.symbol == "BTC/USDT:USDT", _db.timeframe == "1m", _db.timestamp.between(start+2*di, end-di)).order_by(_db.timestamp.asc()))

    topq = db.select(db.timestamp).where(db.exchange == "bybit", db.symbol == "BTC/USDT:USDT", db.timeframe == "1m")
    query_next = db.select(subq.c.tms).from_(subq).where(subq.c.ntms.not_in(topq))
    query_prev = db.select(subq.c.tms).from_(subq).where(subq.c.ptms.not_in(topq))
    _next = [("n", i[0]) for i in list(query_next.tuples())]
    _prev = [("p",i[0]) for i in list(query_prev.tuples())]
    ls = _next + _prev + [("s", start), ("e", end)]
    return sorted(ls, key= lambda x: x[1])
    # gaps = _gaps.extend([("s", start), ("e", end)])
    # sorted(gaps, key=lambda x: x[1])

In [108]:
import pandas as pd
start, end = timly.safe_tms("2022-09-22"), timly.safe_tms("2022-09-24")
res = find_gaps_sorted(db, start, end)


d = {(e[0], res[i+1][0]):(e[1], res[i+1][1]) for i, e in enumerate(res[:-1])}


In [114]:
def match_bounds(bounds: dict) -> tuple[list, list]:
    has, has_not = [], []
    for key, value in bounds.items():
        match key:
            case ("s", "p"):
                return "fetch"
            case ("n", "e"):
                return "fetch"
            case ("p", "n"):
                return "get"
            case ("p", "e"):
                return "get"
            case ("s", "n"):
                return "get"
            case ("s", "e"):
                return "get"



In [112]:
d

{('s', 'e'): (1663804800000, 1663977600000)}

In [115]:
for key, value in d.items():
    print(route_gap_items(key, value))

gets


In [70]:
def find_sorted_gaps(start, end)
ls = [("s", start), ("e", end)]
for n in resn:
    ls.append(("n", n))
for p in resp:
    ls.append(("p", p))
sorted(ls, key=lambda x: x[1])


[('s', 1663632000000), ('e', 1663977600000), ('p', 1663804860000)]

[('s', 1663632000000), ('p', 1663804860000), ('e', 1663977600000)]

In [53]:
res = _find_gaps(db, timly.safe_tms("2022-09-20"), timly.safe_tms("2022-09-24"))
res = [str(pd.to_datetime(i[0], unit="ms")) for i in res]
print(res)

['2022-09-22 00:01:00',
 '2022-09-26 00:00:00',
 '2022-09-30 15:41:00',
 '2022-12-09 00:00:00']

# Create Test Database

In [1]:
import peewee
from Backtester.database.clients.postgres import database

if database.is_closed():
    database.open_connection()

class Test(peewee.Model):
    id = peewee.UUIDField(primary_key=True)
    tms = peewee.IntegerField()
    sym = peewee.CharField()

    # partial candles: 5 * 1m candle = 5m candle while 1m == partial candle
    is_partial = True

    class Meta:
        from Backtester.database.clients.postgres import database

        database = database.db
        indexes = (
            (("tms", "sym"), True),
        )

    def __init__(self, attributes: dict = None, **kwargs) -> None:
        peewee.Model.__init__(self, attributes=attributes, **kwargs)

        if attributes is None:
            attributes = {}

        for a, value in attributes.items():
            setattr(self, a, value)

Test.create_table()

In [3]:
database.open_connection()

In [11]:
database.db.table_exists("test")

True

In [19]:
print(list(Test.select().tuples()))

[(UUID('3a513a10-be5e-4e6d-94b7-3cca7d09e85a'), 1, 'eth'), (UUID('0ee06d43-52e3-45af-aaf7-2a22f6668c1d'), 2, 'eth'), (UUID('1a2d3609-45c7-48d3-8c39-77657652412a'), 3, 'eth'), (UUID('e1d09c27-f60a-4a44-bd03-abceae77f174'), 4, 'eth'), (UUID('c2ffa1c7-d15c-4b23-88b0-3651505a2514'), 5, 'eth'), (UUID('1fda0309-d08b-4abb-87d6-bc3138370bf6'), 6, 'eth'), (UUID('6048256c-6fef-4b6a-b17e-e5a62ccf9f11'), 7, 'eth'), (UUID('4fa97dfc-605b-4f06-bac3-ec4d6f17a9df'), 8, 'eth'), (UUID('dc6ac946-9cf8-4d71-a39a-4c464101cf78'), 9, 'eth'), (UUID('aa113e59-6251-40d4-9b22-23c0ade622da'), 15, 'eth'), (UUID('5a119a76-9790-4a79-9b22-eee1c6377732'), 16, 'eth'), (UUID('a22d111f-9f91-417a-828b-ecac71144dff'), 17, 'eth'), (UUID('4bfe3bdb-7b0f-4bc6-996e-1a52710960bb'), 18, 'eth'), (UUID('48df02fe-d368-4a94-971f-876bd4ab23f6'), 19, 'eth')]


In [4]:
Test.create_table()

ImproperlyConfigured: database attribute does not appear to be set on the model: <Model: Test>

In [16]:
import uuid

def gen_entries(ls):
    ds = []
    for i in ls:
        d = {"id":uuid.uuid4(),
             "tms": i,
             "sym":"eth"}
        ds.append(d)
    return ds

ds1 = gen_entries(range(1, 10))
ds2 = gen_entries(range(20, 22))

Test.insert_many(ds2).on_conflict_ignore().execute()

InterfaceError: Query must be bound to a database in order to call "execute".

In [4]:
query = Test.select()

for q in query:
    print(q.tms)

InterfaceError: Query must be bound to a database in order to call "execute".

In [11]:
Test.insert_many(ds2).on_conflict_ignore().execute()

In [25]:
_Test = Test.alias()

ntms = _Test.tms + 1
ptms = _Test.tms - 1
subq = (_Test.select(_Test.tms.alias("tms"), ntms.alias("ntms"), ptms.alias("ptms")).where(_Test.tms.between(2, 18)))

topq = Test.select(Test.tms)
query = Test.select(subq.c.tms).from_(subq).where((subq.c.ntms.not_in(topq)) | (subq.c.ptms.not_in(topq)))

In [ ]:
def find_gaps(db, start, end, di = 60000):
    _db = db.alias()
    ntms = _db.timestamp + di
    ptms = _db.timestamp - di
    subq = (_db.select(_db.timestamp.alias("tms"), ntms.alias("ntms"), ptms.alias("ptms")).where(_db.timestamp.between(start+di, end-di)))

    topq = db.select(db.timestamp)
    query = db.select(subq.c.tms).from_(subq).where((subq.c.ntms.not_in(topq)) | (subq.c.ptms.not_in(topq)))
    return list(query.tuples())


In [5]:
def p(query):
    return list(query.tuples())

In [8]:
p(subq)

[(1, 2),
 (2, 3),
 (3, 4),
 (4, 5),
 (5, 6),
 (6, 7),
 (7, 8),
 (8, 9),
 (9, 10),
 (15, 16),
 (16, 17),
 (17, 18),
 (18, 19),
 (19, 20)]

In [26]:
topq = Test.select(Test.tms)
query = Test.select(subq.c.tms).from_(subq).where((subq.c.ntms.not_in(topq)) | (subq.c.ptms.not_in(topq)))

In [27]:
p(query)

[(9,), (15,)]

In [17]:
topq = Test.select(Test.tms).where(Test.tms.not_in(subq2))

In [22]:
ntms = _Test.tms + 1

In [21]:
list(query.select().where(_Test.tms.not_in(Test.tms)).tuples())

ValueError: Query has not been executed.